In [1]:
#Tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
#Importing libraries

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications.efficientnet import EfficientNetB0
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [3]:
#Datasets path

#17000 samples original dataset

train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/Original_graphs_12_leads_12700/Training/'
val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/Original_graphs_12_leads_12700/Validation/'

In [4]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 8877 files belonging to 5 classes.


2023-02-17 13:25:17.177715: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 13:25:17.203809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 13:25:17.204128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 13:25:17.234313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 3813 files belonging to 5 classes.


In [5]:
#Normalization of the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [6]:
# Instantiate base_model with pre-trained weights

#import tensorflow.python.keras.applications.VGG16

base_model= tf.keras.applications.VGG16(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [7]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [8]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [9]:
x= base_model(inputs,training= False)

In [10]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [11]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [12]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [13]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [14]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [15]:
#callback

class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [16]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-02-17 13:25:22.716226: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-17 13:25:22.938541: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


70/70 [==============================] - ETA: 0s - loss: 1.1096 - accuracy: 0.5856 - sensitivity_at_specificity: 0.9100 - specificity_at_sensitivity: 0.9287 - recall: 0.4922 - precision: 0.6458
Epoch 1: val_accuracy improved from -inf to 0.48518, saving model to ECG_Spectrogram_Model.h5
70/70 [==============================] - 77s 949ms/step - loss: 1.1096 - accuracy: 0.5856 - sensitivity_at_specificity: 0.9100 - specificity_at_sensitivity: 0.9287 - recall: 0.4922 - precision: 0.6458 - val_loss: 1.1963 - val_accuracy: 0.4852 - val_sensitivity_at_specificity: 0.9156 - val_specificity_at_sensitivity: 0.8720 - val_recall: 0.2709 - val_precision: 0.5486
Epoch 2/50
70/70 [==============================] - ETA: 0s - loss: 0.9556 - accuracy: 0.6411 - sensitivity_at_specificity: 0.9477 - specificity_at_sensitivity: 0.9552 - recall: 0.5505 - precision: 0.7092
Epoch 2: val_accuracy improved from 0.48518 to 0.66037, saving model to ECG_Spectrogram_Model.h5
70/70 [==============================] -

Epoch 14/50
70/70 [==============================] - ETA: 0s - loss: 0.7789 - accuracy: 0.7209 - sensitivity_at_specificity: 0.9756 - specificity_at_sensitivity: 0.9791 - recall: 0.6491 - precision: 0.7809
Epoch 14: val_accuracy improved from 0.71099 to 0.71309, saving model to ECG_Spectrogram_Model.h5
70/70 [==============================] - 59s 838ms/step - loss: 0.7789 - accuracy: 0.7209 - sensitivity_at_specificity: 0.9756 - specificity_at_sensitivity: 0.9791 - recall: 0.6491 - precision: 0.7809 - val_loss: 0.7896 - val_accuracy: 0.7131 - val_sensitivity_at_specificity: 0.9683 - val_specificity_at_sensitivity: 0.9788 - val_recall: 0.6237 - val_precision: 0.7956
Epoch 15/50
70/70 [==============================] - ETA: 0s - loss: 0.7732 - accuracy: 0.7218 - sensitivity_at_specificity: 0.9708 - specificity_at_sensitivity: 0.9794 - recall: 0.6515 - precision: 0.7833
Epoch 15: val_accuracy improved from 0.71309 to 0.72200, saving model to ECG_Spectrogram_Model.h5
70/70 [===============

Epoch 27/50
70/70 [==============================] - ETA: 0s - loss: 0.7122 - accuracy: 0.7430 - sensitivity_at_specificity: 0.9808 - specificity_at_sensitivity: 0.9850 - recall: 0.6826 - precision: 0.7979
Epoch 27: val_accuracy did not improve from 0.73328
70/70 [==============================] - 58s 828ms/step - loss: 0.7122 - accuracy: 0.7430 - sensitivity_at_specificity: 0.9808 - specificity_at_sensitivity: 0.9850 - recall: 0.6826 - precision: 0.7979 - val_loss: 0.7678 - val_accuracy: 0.7257 - val_sensitivity_at_specificity: 0.9719 - val_specificity_at_sensitivity: 0.9826 - val_recall: 0.6800 - val_precision: 0.7690
Epoch 28/50
70/70 [==============================] - ETA: 0s - loss: 0.7123 - accuracy: 0.7459 - sensitivity_at_specificity: 0.9802 - specificity_at_sensitivity: 0.9841 - recall: 0.6829 - precision: 0.7981
Epoch 28: val_accuracy did not improve from 0.73328
70/70 [==============================] - 58s 826ms/step - loss: 0.7123 - accuracy: 0.7459 - sensitivity_at_specifi

Epoch 40/50
70/70 [==============================] - ETA: 0s - loss: 0.6557 - accuracy: 0.7637 - sensitivity_at_specificity: 0.9865 - specificity_at_sensitivity: 0.9878 - recall: 0.7088 - precision: 0.8153
Epoch 40: val_accuracy did not improve from 0.73958
70/70 [==============================] - 58s 827ms/step - loss: 0.6557 - accuracy: 0.7637 - sensitivity_at_specificity: 0.9865 - specificity_at_sensitivity: 0.9878 - recall: 0.7088 - precision: 0.8153 - val_loss: 0.7949 - val_accuracy: 0.7215 - val_sensitivity_at_specificity: 0.9727 - val_specificity_at_sensitivity: 0.9798 - val_recall: 0.6874 - val_precision: 0.7569
Epoch 41/50
70/70 [==============================] - ETA: 0s - loss: 0.6828 - accuracy: 0.7599 - sensitivity_at_specificity: 0.9836 - specificity_at_sensitivity: 0.9850 - recall: 0.7005 - precision: 0.8098
Epoch 41: val_accuracy did not improve from 0.73958
70/70 [==============================] - 58s 827ms/step - loss: 0.6828 - accuracy: 0.7599 - sensitivity_at_specifi

In [17]:
#Acquiring Performance metrics

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.7305215001106262
Validation Accuracy:  0.684688150882721
Validation Specificity:  0.9713265299797058
Validation Sensitivity:  0.9683336615562439
Validation Recall:  0.6327476501464844
Validation Precision:  0.7468982338905334
Validation Loss:  0.8785406947135925
